In [1]:
from src.datasets import SIFT
from src.toolkit import network
import pandas as pd 
import torch
import numpy as np

from src.toolkit import learn

from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import faiss

In [2]:
train_set, base_set, eval_set = SIFT.get_datasets()

loading dataset from /home/jiajunb/neural-dimension-reduction/data/sift/siftsmall


In [3]:
model = network.SiameseNet(network.EmbeddingNet())
learning_rate = 1e-3
batch_size = 524288
num_epoches = 30

verbose = True
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
weight_decay = 1e-6
log_epoch = 1

In [4]:
# optimizer
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(
        nd in n for nd in no_decay) and p.requires_grad], 'weight_decay': weight_decay},
    {'params': [p for n, p in model.named_parameters() if any(
        nd in n for nd in no_decay) and p.requires_grad], 'weight_decay': 0.0}
]

optimizer = torch.optim.AdamW(params=optimizer_grouped_parameters, lr=learning_rate)

train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size, pin_memory=True)
base_loader = DataLoader(base_set, shuffle=False, batch_size=batch_size, pin_memory=True)
eval_loader = DataLoader(eval_set, shuffle=False, batch_size=batch_size, pin_memory=True)

model = model.to(device)

In [5]:
criterion = learn.PowerMarginLoss(10, 'mean')

In [ ]:
best_recall_query_set, its_recall_on_base_set, best_model, model = learn.train_with_eval(train_loader, base_loader, eval_loader, criterion, model, optimizer, num_epoches, log_epoch,
                    verbose, device)

training loss: 63.335884
epoch [1]/[30] training loss: 0.000122 recall on query set: 0.03 recall on base set:  0.04
training loss: 60.699104
epoch [2]/[30] training loss: 0.000120 recall on query set: 0.03 recall on base set:  0.04
training loss: 60.021721
epoch [3]/[30] training loss: 0.000118 recall on query set: 0.03 recall on base set:  0.04
training loss: 59.415558
epoch [4]/[30] training loss: 0.000117 recall on query set: 0.03 recall on base set:  0.04


In [ ]:
best_recall_query_set

In [ ]:
its_recall_on_base_set